<a href="https://colab.research.google.com/github/changedi/DPpro/blob/master/semantic_router_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tzdata
!pip install -qU semantic-router

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [2]:
from semantic_router import Route

politics = Route(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president" "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)
chitchat = Route(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

routes = [politics, chitchat]

In [3]:
import os
from getpass import getpass
from semantic_router import RouteLayer
from semantic_router.encoders import CohereEncoder, OpenAIEncoder

# dashboard.cohere.ai
# os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY") or getpass(
#     "Enter Cohere API Key: "
# )
# platform.openai.com
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)

# encoder = CohereEncoder()
encoder = OpenAIEncoder()

rl = RouteLayer(encoder=encoder, routes=routes)

Enter OpenAI API Key: ··········


In [4]:
rl("how's the weather today?")


RouteChoice(name='chitchat', function_call=None, similarity_score=None)

# dynamic rule


In [5]:
from datetime import datetime
from zoneinfo import ZoneInfo


def get_time(timezone: str) -> str:
    """Finds the current time in a specific timezone.

    :param timezone: The timezone to find the current time in, should
        be a valid timezone from the IANA Time Zone Database like
        "America/New_York" or "Europe/London". Do NOT put the place
        name itself like "rome", or "new york", you must provide
        the IANA format.
    :type timezone: str
    :return: The current time in the specified timezone."""
    now = datetime.now(ZoneInfo(timezone))
    return now.strftime("%H:%M")

In [ ]:
get_time("America/New_York")


In [8]:
from semantic_router.llms.openai import get_schemas_openai

schemas = get_schemas_openai([get_time])
schemas

[{'type': 'function',
  'function': {'name': 'get_time',
   'description': 'Finds the current time in a specific timezone.\n\n:param timezone: The timezone to find the current time in, should\n    be a valid timezone from the IANA Time Zone Database like\n    "America/New_York" or "Europe/London". Do NOT put the place\n    name itself like "rome", or "new york", you must provide\n    the IANA format.\n:type timezone: str\n:return: The current time in the specified timezone.',
   'parameters': {'type': 'object',
    'properties': {'timezone': {'type': 'string',
      'description': 'The timezone to find the current time in, should\n    be a valid timezone from the IANA Time Zone Database like\n    "America/New_York" or "Europe/London". Do NOT put the place\n    name itself like "rome", or "new york", you must provide\n    the IANA format.'}},
    'required': ['timezone']}}}]

In [12]:
time_route = Route(
    name="get_time",
    utterances=[
        "what is the time in new york city?",
        "what is the time in london?",
        "I live in Rome, what time is it?",
    ],
    function_schemas=schemas,
)

In [13]:
time_route.llm


In [14]:
rl.add(time_route)


In [15]:
time_route.llm


In [16]:
response = rl("what is the time in new york city?")
response

RouteChoice(name='get_time', function_call=[{'function_name': 'get_time', 'arguments': {'timezone': 'America/New_York'}}], similarity_score=None)

In [17]:
print(response.function_call)


[{'function_name': 'get_time', 'arguments': {'timezone': 'America/New_York'}}]


In [18]:
import json

for call in response.function_call:
    if call["function_name"] == "get_time":
        args = call["arguments"]
        result = get_time(**args)
print(result)

23:55


## dynamic rules with multiple functions

In [19]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo


# Function with one argument
def get_time(timezone: str) -> str:
    """Finds the current time in a specific timezone.

    :param timezone: The timezone to find the current time in, should
        be a valid timezone from the IANA Time Zone Database like
        "America/New_York" or "Europe/London". Do NOT put the place
        name itself like "rome", or "new york", you must provide
        the IANA format.
    :type timezone: str
    :return: The current time in the specified timezone."""
    now = datetime.now(ZoneInfo(timezone))
    return now.strftime("%H:%M")


def get_time_difference(timezone1: str, timezone2: str) -> str:
    """Calculates the time difference between two timezones.
    :param timezone1: The first timezone, should be a valid timezone from the IANA Time Zone Database like "America/New_York" or "Europe/London".
    :param timezone2: The second timezone, should be a valid timezone from the IANA Time Zone Database like "America/New_York" or "Europe/London".
    :type timezone1: str
    :type timezone2: str
    :return: The time difference in hours between the two timezones."""
    # Get the current time in UTC
    now_utc = datetime.utcnow().replace(tzinfo=ZoneInfo("UTC"))

    # Convert the UTC time to the specified timezones
    tz1_time = now_utc.astimezone(ZoneInfo(timezone1))
    tz2_time = now_utc.astimezone(ZoneInfo(timezone2))

    # Calculate the difference in offsets from UTC
    tz1_offset = tz1_time.utcoffset().total_seconds()
    tz2_offset = tz2_time.utcoffset().total_seconds()

    # Calculate the difference in hours
    hours_difference = (tz2_offset - tz1_offset) / 3600

    return f"The time difference between {timezone1} and {timezone2} is {hours_difference} hours."


# Function with three arguments
def convert_time(time: str, from_timezone: str, to_timezone: str) -> str:
    """Converts a specific time from one timezone to another.
    :param time: The time to convert in HH:MM format.
    :param from_timezone: The original timezone of the time, should be a valid IANA timezone.
    :param to_timezone: The target timezone for the time, should be a valid IANA timezone.
    :type time: str
    :type from_timezone: str
    :type to_timezone: str
    :return: The converted time in the target timezone.
    :raises ValueError: If the time format or timezone strings are invalid.

    Example:
        convert_time("12:30", "America/New_York", "Asia/Tokyo") -> "03:30"
    """
    try:
        # Use today's date to avoid historical timezone issues
        today = datetime.now().date()
        datetime_string = f"{today} {time}"
        time_obj = datetime.strptime(datetime_string, "%Y-%m-%d %H:%M").replace(
            tzinfo=ZoneInfo(from_timezone)
        )

        converted_time = time_obj.astimezone(ZoneInfo(to_timezone))

        formatted_time = converted_time.strftime("%H:%M")
        return formatted_time
    except Exception as e:
        raise ValueError(f"Error converting time: {e}")

In [21]:
functions = [get_time, get_time_difference, convert_time]
# Generate schemas for all functions
from semantic_router.llms.openai import get_schemas_openai

schemas = get_schemas_openai(functions)
schemas

[{'type': 'function',
  'function': {'name': 'get_time',
   'description': 'Finds the current time in a specific timezone.\n\n:param timezone: The timezone to find the current time in, should\n    be a valid timezone from the IANA Time Zone Database like\n    "America/New_York" or "Europe/London". Do NOT put the place\n    name itself like "rome", or "new york", you must provide\n    the IANA format.\n:type timezone: str\n:return: The current time in the specified timezone.',
   'parameters': {'type': 'object',
    'properties': {'timezone': {'type': 'string',
      'description': 'The timezone to find the current time in, should\n    be a valid timezone from the IANA Time Zone Database like\n    "America/New_York" or "Europe/London". Do NOT put the place\n    name itself like "rome", or "new york", you must provide\n    the IANA format.'}},
    'required': ['timezone']}}},
 {'type': 'function',
  'function': {'name': 'get_time_difference',
   'description': 'Calculates the time differ

In [22]:
# Define the dynamic route with multiple functions
multi_function_route = Route(
    name="timezone_management",
    utterances=[
        # Utterances for get_time function
        "what is the time in New York?",
        "current time in Berlin?",
        "tell me the time in Moscow right now",
        "can you show me the current time in Tokyo?",
        "please provide the current time in London",
        # Utterances for get_time_difference function
        "how many hours ahead is Tokyo from London?",
        "time difference between Sydney and Cairo",
        "what's the time gap between Los Angeles and New York?",
        "how much time difference is there between Paris and Sydney?",
        "calculate the time difference between Dubai and Toronto",
        # Utterances for convert_time function
        "convert 15:00 from New York time to Berlin time",
        "change 09:00 from Paris time to Moscow time",
        "adjust 20:00 from Rome time to London time",
        "convert 12:00 from Madrid time to Chicago time",
        "change 18:00 from Beijing time to Los Angeles time"
        # All three functions
        "What is the time in Seattle? What is the time difference between Mumbai and Tokyo? What is 5:53 Toronto time in Sydney time?",
    ],
    function_schemas=schemas,
)
routes = [politics, chitchat, multi_function_route]
rl2 = RouteLayer(encoder=encoder, routes=routes)


In [23]:
def parse_response(response: str):

    for call in response.function_call:
        args = call["arguments"]
        if call["function_name"] == "get_time":
            result = get_time(**args)
            print(result)
        if call["function_name"] == "get_time_difference":
            result = get_time_difference(**args)
            print(result)
        if call["function_name"] == "convert_time":
            result = convert_time(**args)
            print(result)

In [24]:
response = rl2("What is your political leaning?")
response

RouteChoice(name='politics', function_call=None, similarity_score=None)

In [25]:
response = rl2("Hello bot, how are you today?")
response

RouteChoice(name='chitchat', function_call=None, similarity_score=None)

In [26]:
response = rl2("what is the time in New York?")
response

RouteChoice(name='timezone_management', function_call=[{'function_name': 'get_time', 'arguments': {'timezone': 'America/New_York'}}], similarity_score=None)

In [27]:
parse_response(response)


23:58


In [28]:
response = rl2("What is the time difference between Los Angeles and Istanbul?")
response

RouteChoice(name='timezone_management', function_call=[{'function_name': 'get_time_difference', 'arguments': {'timezone1': 'America/Los_Angeles', 'timezone2': 'Europe/Istanbul'}}], similarity_score=None)

In [29]:
parse_response(response)

The time difference between America/Los_Angeles and Europe/Istanbul is 10.0 hours.


In [30]:
response = rl2("What is 23:02 Dubai time in Tokyo time? Please and thank you.")
response

RouteChoice(name='timezone_management', function_call=[{'function_name': 'convert_time', 'arguments': {'time': '23:02', 'from_timezone': 'Asia/Dubai', 'to_timezone': 'Asia/Tokyo'}}], similarity_score=None)

In [31]:
parse_response(response)


04:02


In [32]:
response = rl2(
    """
    What is the time in Prague?
    What is the time difference between Frankfurt and Beijing?
    What is 5:53 Lisbon time in Bangkok time?
"""
)

In [33]:
response

RouteChoice(name='timezone_management', function_call=[{'function_name': 'get_time', 'arguments': {'timezone': 'Europe/Prague'}}, {'function_name': 'get_time_difference', 'arguments': {'timezone1': 'Europe/Berlin', 'timezone2': 'Asia/Shanghai'}}, {'function_name': 'convert_time', 'arguments': {'time': '05:53', 'from_timezone': 'Europe/Lisbon', 'to_timezone': 'Asia/Bangkok'}}], similarity_score=None)

In [34]:
parse_response(response)


06:00
The time difference between Europe/Berlin and Asia/Shanghai is 6.0 hours.
11:53


# save & load json

In [35]:
rl.to_json("layer.json")


In [36]:
import json

with open("layer.json", "r") as f:
    layer_json = json.load(f)

print(layer_json)

{'encoder_type': 'openai', 'encoder_name': 'text-embedding-ada-002', 'routes': [{'name': 'politics', 'utterances': ["isn't politics the best thing ever", "why don't you tell me about your political opinions", "don't you just love the presidentdon't you just hate the president", "they're going to destroy this country!", 'they will save the country!'], 'description': None, 'function_schemas': None, 'llm': None, 'score_threshold': 0.82}, {'name': 'chitchat', 'utterances': ["how's the weather today?", 'how are things going?', 'lovely weather today', 'the weather is horrendous', "let's go to the chippy"], 'description': None, 'function_schemas': None, 'llm': None, 'score_threshold': 0.82}, {'name': 'get_time', 'utterances': ['what is the time in new york city?', 'what is the time in london?', 'I live in Rome, what time is it?'], 'description': None, 'function_schemas': [{'type': 'function', 'function': {'name': 'get_time', 'description': 'Finds the current time in a specific timezone.\n\n:p

In [37]:
rl = RouteLayer.from_json("layer.json")
print(
    f"""{rl.encoder.type=}
{rl.encoder.name=}
{rl.routes=}"""
)

rl.encoder.type='openai'
rl.encoder.name='text-embedding-ada-002'
rl.routes=[Route(name='politics', utterances=["isn't politics the best thing ever", "why don't you tell me about your political opinions", "don't you just love the presidentdon't you just hate the president", "they're going to destroy this country!", 'they will save the country!'], description=None, function_schemas=None, llm=None, score_threshold=0.82), Route(name='chitchat', utterances=["how's the weather today?", 'how are things going?', 'lovely weather today', 'the weather is horrendous', "let's go to the chippy"], description=None, function_schemas=None, llm=None, score_threshold=0.82), Route(name='get_time', utterances=['what is the time in new york city?', 'what is the time in london?', 'I live in Rome, what time is it?'], description=None, function_schemas=[{'type': 'function', 'function': {'name': 'get_time', 'description': 'Finds the current time in a specific timezone.\n\n:param timezone: The timezone to find 

In [38]:
rl("isn't politics the best thing ever")


RouteChoice(name='politics', function_call=None, similarity_score=None)